In [1]:
!git clone https://github.com/maria-natale/ProgettoFVAB

"git" non Š riconosciuto come comando interno o esterno,
 un programma eseguibile o un file batch.


In [9]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
!python "/content/ProgettoFVAB/Code/create_dataset.py"

file_dataset/spagnolo_giapponese.csv
100% 201/201 [01:32<00:00,  2.17it/s]
100% 55/55 [00:24<00:00,  2.28it/s]


In [78]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing import image
from tqdm import tqdm
from PIL import Image
import gc

path_git = '/content/ProgettoFVAB'
filename = '4_7'
path_drive = 'C:/Users/CasaLab1/Desktop/Casillo&Natale/DATASET'
dataset_dir = 'dataset_4_7'
"""LANGUAGES = {
  1:'Italiano',
  2:'Inglese',
  3: 'Tedesco',
  4:'Spagnolo',
  5: 'Olandese',
  6:'Russo',
  7: 'Giapponese'}
LANGUAGES_N = {
    1:0,
    2:1,
    3:2,
    4:3,
    5:4,
    6:5,
    7:6
}"""
LANGUAGES = {
    4:'Spagnolo',
    7: 'Giapponese'}
LANGUAGES_N = {
    4:0,
    7:1
}

In [5]:
!pip install keras-video-generators

  Created wheel for keras-video-generators: filename=keras_video_generators-1.0.14-cp37-none-any.whl size=12884 sha256=a71b5c8b3d31823aabbb9801b61237e92c82670b21b7f028a1e1b976f8d6f6af
  Stored in directory: /root/.cache/pip/wheels/20/b7/76/8674d46fc4777c09e5aa7b065d4e356d90f12ec409a6144bbb
Successfully built keras-video-generators


In [9]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


SystemError: GPU device not found

In [10]:
import tensorflow as tf
tf.config.set_soft_device_placement(True)
tf.debugging.set_log_device_placement(True)

# Creates some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [79]:
SIZE = (64, 64)
CHANNELS = 3
NBFRAME = 350
BS = 4

import glob
import keras
import keras_video
import tensorflow

def load_video(path, group):
  # use sub directories names as classes
  os.chdir(os.path.join(path, group))
  classes = [i.split(os.path.sep)[-1] for i in glob.glob(os.path.join(path, group)+'/*')]
  print(classes)
  classes.sort()
  # some global params
  # pattern to get videos and classes
  glob_pattern=path+'/'+group+'/{classname}/*.avi'
  # for data augmentation
  data_aug = tensorflow.keras.preprocessing.image.ImageDataGenerator(
      zoom_range=.1,
      horizontal_flip=True,
      rotation_range=8,
      width_shift_range=.2,
      height_shift_range=.2)
  # Create video frame generator
  x = keras_video.VideoFrameGenerator(
      #sequence_time=1,
      classes=classes, 
      nb_frames=NBFRAME,
      glob_pattern=glob_pattern,
      shuffle=True,
      batch_size=BS,
      target_shape=SIZE,
      nb_channel=CHANNELS,
      use_frame_cache=False
      #transformation=data_aug
  )
  return x

NON ESEGUIRE. CAMBIA STI PATH 

In [80]:
train = load_video(os.path.join(path_drive, dataset_dir), 'train')
validation = load_video(os.path.join(path_drive, dataset_dir), 'validation')
test = load_video(os.path.join(path_drive, dataset_dir), 'test')

['0', '1']
Total data: 2 classes for 160 files for train
['0', '1']
Total data: 2 classes for 43 files for train
['0', '1']
Total data: 2 classes for 50 files for train


In [81]:
len(train), len(test)

(40, 12)

AttributeError: 'VideoFrameGenerator' object has no attribute 'vid_inf'

In [60]:
from keras_video import utils as ku
ku.show_sample(test)

AssertionError: ignored

In [ ]:
ku.show_sample(train, index=1)

In [ ]:
ku.show_sample(train)




In [ ]:
from keras_video import utils as ku
ku.show_sample(train)


In [ ]:
from google.colab.patches import cv2_imshow
from IPython.display import Image
for i in train: 
  
  c=i
  break
print(i)


In [82]:
#tf.device('/device:GPU:0')
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, GlobalMaxPool2D
def build_convnet(shape=(112, 112, 3)):
    momentum = .9
    model = Sequential()
    model.add(Conv2D(64, (3,3), input_shape=shape, padding='same', activation='relu'))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    # flatten...
    model.add(GlobalMaxPool2D())
    return model

In [83]:
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import TimeDistributed, Dense, GRU, Dropout
def action_model(shape=(5, 112, 112, 3), nbout=2):
    # Create our convnet with (112, 112, 3) input shape
    convnet = build_convnet(shape[1:])
    
    # then create our final model
    model = Sequential()
    # add the convnet with (5, 112, 112, 3) shape
    model.add(TimeDistributed(convnet, input_shape=shape))
    model.add(Bidirectional(LSTM(units = 100, return_sequences = True, input_shape = (NBFRAME, 64*64*3))))
    model.add(Dropout(0.8))

    # Adding a second LSTM layer and Dropout layer
    model.add(Bidirectional(LSTM(units = 100, return_sequences = True)))
    model.add(Dropout(0.8))

    # Adding a third LSTM layer and Dropout layer
    model.add(Bidirectional(LSTM(units = 100)))
    model.add(Dropout(0.8))
    model.add(Dense(nbout, activation='softmax'))
    model.summary()
    return model

In [84]:

INSHAPE=(NBFRAME,) + SIZE + (CHANNELS,) # (5, 112, 112, 3)
model = action_model(INSHAPE, len(LANGUAGES_N.keys()))
optimizer = tensorflow.keras.optimizers.Adam(0.001)
model.compile(
    optimizer,
    'binary_crossentropy',
    metrics=['acc']
)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 350, 512)          4689216   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 350, 200)          490400    
_________________________________________________________________
dropout_3 (Dropout)          (None, 350, 200)          0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 350, 200)          240800    
_________________________________________________________________
dropout_4 (Dropout)          (None, 350, 200)          0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 200)               240800    
_________________________________________________________________
dropout_5 (Dropout)          (None, 200)              

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
with tensorflow.device('/device:GPU:0'):
    tensorflow.debugging.set_log_device_placement(True)
    EPOCHS=30
    class MyCustomCallback(tensorflow.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs=None):
            gc.collect()
    
    es = EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=10,
                              verbose=0, mode='auto')

    history =model.fit_generator(
        train,
        validation_data=validation,
        epochs=EPOCHS,
        steps_per_epoch = int(len(train)/BS),
        validation_steps= len(validation), 
        callbacks = [es]
        #callbacks=[MyCustomCallback()]
    )

c:\users\casalab1\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30


In [ ]:

predictions = model.predict_generator(test, steps = len(test))
predictions = predictions.argmax(axis = -1)
predictions

In [ ]:
y_true =[]
x,y = test.next()
for i in range(0,len(test.files)//BS):
    for j in range(BS):
        y_true.append(y[j])
    x,y = test.next()

In [ ]:
y_true = np.array(y_true).argmax(axis = -1)

In [ ]:
from sklearn.metrics import classification_report

target_names = list(LANGUAGES.values())
print(classification_report(y_true, predictions, target_names=target_names))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

valori=pd.crosstab(predictions, y_true)
fig=plt.figure(figsize=(17,5))
ax1=plt.subplot(121)
ax1.set_title("spagnolo e giapponese")
sns.heatmap(valori,annot=True,cmap="Blues",cbar=False)
plt.xlabel("attuali") 
plt.ylabel("predetti")


In [ ]:
for j in (list(LANGUAGES_N.keys())):
  i = LANGUAGES_N[j]
  TP= valori.iloc[i,i]
  FP= valori.iloc[i,:].sum()-TP
  FN= valori.iloc[:,i].sum()-TP
  TN= valori.sum().sum() - TP - FP - FN
  print(TP,FP,FN,TN)
  #b=[[TP,FP],[FN,TN]]
  b=[[TP,TN],[FP,FN]]
  fig=plt.figure(figsize=(17,5))
  ax1=plt.subplot(121)
  sns.heatmap(b,annot=True,cmap="Blues",cbar=False,fmt = "g")
  #plt.xlabel("POSITIVI") 
  #plt.ylabel("NEGATIVI")
  tick_marks = [0.5,1.5]
  plt.title(LANGUAGES[j])
  plt.xticks(tick_marks, ["P","N",])
  plt.yticks(tick_marks, ["T","F"])

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc

vis_arr = multilabel_confusion_matrix(y_true, predictions, labels = list(LANGUAGES_N.values()))
print(vis_arr)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in list(LANGUAGES_N.values()):
    fpr[i], tpr[i], _ = roc_curve(y_true == i, predictions == i)
    roc_auc[i] = auc(fpr[i], tpr[i])


In [ ]:
# Plot of a ROC curve for a specific class
for i in list(LANGUAGES_N.values()):
    plt.figure()
    plt.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f)' % roc_auc[i])
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()